# Programa para generar Casos de Cursogramas utilizados para el entrenamiento, validación y prueba del Modelo de Detección de Objetos

1) Importar Librerias

In [ ]:
#@title Cargar las librerías necesarias

import os
import os.path
import json
import math
import string
import numpy as np
import xml.etree.cElementTree as ET

from os.path import isfile, join
from PIL import Image, ImageDraw, ImageFont
from random import choice, choices, randint, shuffle, sample

import csv

print ("Librerías cargadas.")

2) Montar Drive:

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

3) Configuración:

In [ ]:
#@title Configuración de Parámetros


#@markdown Parámetros Generales:
cantidad_casos_generar_total = 145 #@param {type:"slider", min:1, max:5000, step:10}
prefijo_inicial_caso_generado =  "AERS" #@param {type:"string"}
forzar_generar_combinaciones_parametros = False #@param {type:"boolean"}

# parámetros para tamaños de casos
box_size = (100, 100)
page_size = (794, 1123)
white_line_size = 10

#@markdown Parámetros generales sobre generación de símbolos en los casos:
generar_errores = False 
usar_config_probabilidad_simbolos = False #@param {type:"boolean"}
generar_simbolos_sin_referencias = False #@param {type:"boolean"}
usar_config_valores_tipo_referencias = True #@param {type:"boolean"}
usar_referencias_generadas = "SECUENCIAL_TIPO_REFERENCIA" #@param ["AZAR", "SECUENCIAL_TIPO_REFERENCIA", "SECUENCIAL_SIMBOLO"]

#@markdown Parámetros especiales sobre generación forzada de símbolos en los casos:
forzar_decision = True #@param {type:"boolean"}
forzar_copia = False #@param {type:"boolean"}
forzar_area_externa = True #@param {type:"boolean"}
forzar_actualizacion = False #@param {type:"boolean"}

#@markdown Parámetros sobre manejo del Drive:
forzar_drive_borrado = False #@param {type:"boolean"}
forzar_drive_actualizar = True #@param {type:"boolean"}

# ubicación de elementos en el drive
path_base = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Generador-Casos/' #@param {type:"string"}
path_simbolos = path_base + 'simbolos'
path_fuente = path_base + 'fuentes/verdana.ttf'
path_reglas = path_base + 'generador_reglas.json'
path_config = path_base + 'generador_simbolos_config.csv'
path_destino = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/Generados/' #@param {type:"string"}

print("Parámetros definidos.")

# si fuerza generar diferentes combinaciones de parámetros
# determina la cantidad de ciclos a realizar
if forzar_generar_combinaciones_parametros:
  print(" ++ se detecta activación de [forzar_generar_combinaciones_parametros] ")
  print("    por lo que ciertos parámetros se ignoran para generar los casos ++")


In [ ]:
#@title Carga la configuración de los Símbolos y Referencias

# carga la información de un archivo CSV
##model_drive_path
config_simbolos_file = path_config 
with open(config_simbolos_file, mode='r') as csvfile:
    lineasCSV = list(csv.reader(csvfile))

# procesa el archivo CSV
auxListS = []
auxListR = []
for l in lineasCSV:
  # si no se debe ignorar las líneas de comentarios
  if l[0][0] != "#":
    if l[0] == "$":
      # carga la configuración del símbolo
      auxListS.append( ( l[1], [ int(l[2]), int(l[3]), l[4] ] ) )
    elif l[0] == "!":
      # carga la configuración de referencia especial
      auxListR.append( ( l[1], l[2].split("*") ) )

# genera dicccionario de la configuración 
config_simbolos = dict(auxListS)
config_referencias = dict(auxListR)

print("> Configuración cargada de [", config_simbolos_file, "] ")
print("$) Configuración de Símbolos: ", len(auxListS))
print("  valores: nombreSimbolo + [probabilidadUsarSectorInterno, probabilidadUsarSectorExterno, tipoReferencia] ] ")
print("  ", config_simbolos)
print("!) Configuración de Referencias: ", len(auxListR))
print("  valores: tipoReferencia + [valores permitidos] ] ")
print("  ", config_referencias)

# constantes de posicion de configuraciones
config_Probab_Defecto = 50
config_posProbabUsarSectorInterno = 0
config_posProbabUsarSectorExterno = 1
config_posTipoReferencia = 2


4) Definir funciones auxiliares:

In [ ]:
#@title Funciones Generales

def inicializar_carpeta_destino(path_destino, forzar_drive_borrado = False):
  if os.path.isdir(path_destino):
    if forzar_drive_borrado:
      files = os.listdir(path_destino)
      for file in files:
        os.remove(path_destino + file)
    else:
      contar_cantidad_casos(path_destino)
  else:
    os.makedirs(path_destino)
  return 

def cargar_reglas():
  with open(path_reglas) as f:
    reglas = json.load(f)
  return reglas

def crear_cursograma(nombre):
  xml_annotation = generar_documento_xml(nombre)
  page = Image.new('RGB', page_size, (255, 255, 255))
  return page, xml_annotation

def guardar_cursograma(nombre, page, xml_annotation):
  page.save(path_destino + nombre + ".png")
  xml_document = ET.ElementTree(xml_annotation)
  xml_document.write(path_destino + nombre + ".xml" )

def obtener_tamanio_simbolos():
  # obtiene el tamaño real de las imagenes de los símbolos
  dicc = {}
  simbolosDisp = os.listdir(path_simbolos)
  for s in simbolosDisp:
    if s.find(".png") > 0 or s.find(".jpg") > 0:
      img_simbolo = Image.open(join(path_simbolos, s))
      dicc[s] = img_simbolo.size
  return dicc

def contar_cantidad_casos(path_destino):
  # indica cantidad de casos generados
  casosGenerados = os.listdir(path_destino)
  print("\n")
  print("-- Total de casos PNG disponibles: ", len( [ar for ar in casosGenerados if ar.find("png")>0] ))
  print("-- Total de casos XML disponibles: ", len( [ar for ar in casosGenerados if ar.find("xml")>0] ))
  print("\n")
  return

print("Funciones generales definidas.")

In [ ]:
#@title Funciones Especiales para generar XML

def generar_documento_xml(nombre):
  nombre += ".png"
  xml_annotation = ET.Element('annotation')
  ET.SubElement(xml_annotation, 'folder').text = path_destino
  ET.SubElement(xml_annotation, 'filename').text = nombre
  ET.SubElement(xml_annotation, 'path').text = path_destino + nombre

  xml_source = ET.SubElement(xml_annotation, 'source')
  ET.SubElement(xml_source, 'database').text = 'Unknown'

  xml_size = ET.SubElement(xml_annotation, 'size')
  ET.SubElement(xml_size, 'width').text = str(page_size[0])
  ET.SubElement(xml_size, 'height').text = str(page_size[1])
  ET.SubElement(xml_size, 'depth').text = str(3)

  ET.SubElement(xml_annotation, 'segmented').text = '0'

  return xml_annotation

def agregar_anotacion(row, col, xml_annotation, simbolo):
  width, height = Image.open(join(path_simbolos, simbolo)).size
  simbolo = simbolo.replace(".png", "")

  object = ET.SubElement(xml_annotation, 'object')
  ET.SubElement(object, 'name').text = simbolo
  ET.SubElement(object, 'pose').text = 'Unspecified'
  ET.SubElement(object, 'truncated').text = '0'
  ET.SubElement(object, 'difficult').text = '0'

  xmin = int(col * box_size[0]) + int(0.5*(box_size[0] - width))
  ymin = int(row * box_size[1]) + int(0.5*(box_size[1] - height))
  xmax = int(col * box_size[0]) + width + int(0.5*(box_size[0] - width))
  ymax = int(row * box_size[1]) + height + int(0.5*(box_size[1] - height))
  
  if "horizontal" in simbolo:
    ymin -=  20
    ymax += 20

  elif "vertical" in simbolo:
    xmin -= 20
    xmax += 20
  
  xml_bndbox = ET.SubElement(object, 'bndbox')
  ET.SubElement(xml_bndbox, 'xmin').text = str(xmin)
  ET.SubElement(xml_bndbox, 'ymin').text = str(ymin)
  ET.SubElement(xml_bndbox, 'xmax').text = str(xmax)
  ET.SubElement(xml_bndbox, 'ymax').text = str(ymax)

def agregar_anotacion_temporalidad(posicion_y, xml_annotation):
  object = ET.SubElement(xml_annotation, 'object')
  ET.SubElement(object, 'name').text = "temporalidad"
  ET.SubElement(object, 'pose').text = 'Unspecified'
  ET.SubElement(object, 'truncated').text = '0'
  ET.SubElement(object, 'difficult').text = '0'

  xml_bndbox = ET.SubElement(object, 'bndbox')
  ET.SubElement(xml_bndbox, 'xmin').text = str(0)
  ET.SubElement(xml_bndbox, 'ymin').text = str(posicion_y)
  ET.SubElement(xml_bndbox, 'xmax').text = str(page_size[0])
  ET.SubElement(xml_bndbox, 'ymax').text = str(posicion_y + 50)

def agregar_anotacion_transicion_copia(origen, destino, xml_annotation, simbolo="trasladoDeInformacion"):
  
  if "horizontal" in simbolo or "vertical" in simbolo:
    simbolo = simbolo.replace(".png", "")
    xmin = origen[0]
    ymin = origen[1]
    xmax = destino[0]
    ymax = destino[1]

    if "horizontal" in simbolo:  
      ymin -= 20
      ymax += 20
    else: #lif "vertical" in simbolo:
      xmin -= 20
      xmax += 20
  else:
    if origen[1] < destino[1]:
      ymax = destino[1]-5
      ymin = origen[1]-5
    else:
      ymax = origen[1]-5
      ymin = destino[1]-5

    if origen[0] < destino[0]:
      xmax = destino[0]-20
      xmin = origen[0]-20
    else:
      xmax = origen[0]+20
      xmin = destino[0]+20

  object = ET.SubElement(xml_annotation, 'object')
  ET.SubElement(object, 'name').text = simbolo
  ET.SubElement(object, 'pose').text = 'Unspecified'
  ET.SubElement(object, 'truncated').text = '0'
  ET.SubElement(object, 'difficult').text = '0'

  xml_bndbox = ET.SubElement(object, 'bndbox')
  ET.SubElement(xml_bndbox, 'xmin').text = str(xmin)
  ET.SubElement(xml_bndbox, 'ymin').text = str(ymin)
  ET.SubElement(xml_bndbox, 'xmax').text = str(xmax)
  ET.SubElement(xml_bndbox, 'ymax').text = str(ymax)

def agregar_anotacion_area(posicion_x, xml_annotation, area_interna):

  if posicion_x > 0:
    name = "separador_area_interna" if area_interna else "separador_area_externa"
    object = ET.SubElement(xml_annotation, 'object')
    ET.SubElement(object, 'name').text = name
    ET.SubElement(object, 'pose').text = 'Unspecified'
    ET.SubElement(object, 'truncated').text = '0'
    ET.SubElement(object, 'difficult').text = '0'

    xml_bndbox = ET.SubElement(object, 'bndbox')
    ET.SubElement(xml_bndbox, 'xmin').text = str(posicion_x - 10)
    ET.SubElement(xml_bndbox, 'ymin').text = str(0)
    ET.SubElement(xml_bndbox, 'xmax').text = str(posicion_x + 10)
    ET.SubElement(xml_bndbox, 'ymax').text = str(page_size[1])

def marcar_documento_error():
  global nombre, xml_annotation
  nombre += + "_E"
  xml_annotation.find("filename").text = nombre + ".png"
  xml_annotation.find("path").text = path_destino + nombre + ".png"

print("Funciones Especiales para generar XML definidas.")

In [ ]:
#@title Funciones para Referencias de Símbolos 

referencias_valores_numeros_romanos = ["I", "II", "III", "IV", "V", "VI", "VII", "IX", "X"]

aux_anterior_referencia_documento = None

def devolverReferenciaSimbolo(nombreSimbolo):
  global diccReferenciasSecuenciales
  global aux_anterior_referencia_documento

  # si se deben generar simbolos sin referencias
  if generar_simbolos_sin_referencias:
    return "  "

  # determina tipo de referencia del símbolo
  if nombreSimbolo in config_simbolos.keys():
    tipoRef = config_simbolos[nombreSimbolo][config_posTipoReferencia]
  else:
    tipoRef = '-'

  if tipoRef != 'D':
    aux_anterior_referencia_documento = None

  # determina la fuente de información para la referencia
  fuenteRef = ''
  if tipoRef == 'A':
    # Número Arábigo (le saca el '0' inicial para que arranque en '1')
    fuenteRef = string.digits[1:] 
  elif tipoRef == 'R':
    # Número Romano
    fuenteRef = referencias_valores_numeros_romanos
  elif tipoRef == 'L':
    # Letra
    fuenteRef = string.ascii_uppercase
  elif tipoRef == 'D':
    # Letras de tipo Documentos
    # (directamente la devuelve porque no tiene secuencia)
    if usar_config_valores_tipo_referencias and tipoRef in config_referencias.keys():
      return choice(config_referencias[tipoRef])
    else:
      return ''.join(sample(string.ascii_uppercase, randint(1, 2)))
  elif tipoRef == 'T':
    # Textos de Temporalidad
    # (directamente la devuelve porque no tiene secuencia)
    if usar_config_valores_tipo_referencias and tipoRef in config_referencias.keys():
      return choice(config_referencias[tipoRef])
    else:      
      return ''.join(sample(string.ascii_uppercase, randint(4, 10)))
  else:
    # No Aplica o "desconocido"
    # (directamente la devuelve porque no tiene secuencia)
    return ""

  # determina tipo de secuencia de referencia a usar
  if usar_referencias_generadas == "AZAR":
    # devuelve un elemento al azar 
    return choice( fuenteRef ) 
  else:
    # obtiene siguiente elemento a usar por tipo de referencia
    if usar_referencias_generadas == "SECUENCIAL_TIPO_REFERENCIA":
      auxIdRef = "REF_TREF_" + tipoRef
    elif usar_referencias_generadas == "SECUENCIAL_SIMBOLO":
      auxIdRef = "REF_S_" + nombreSimbolo
    else:
      return ""   
    if auxIdRef in diccReferenciasSecuenciales:
      auxPos =  diccReferenciasSecuenciales[auxIdRef] + 1  
    else:
      diccReferenciasSecuenciales[auxIdRef] = 0
      auxPos = 0
    # determina al azar si usa el incremento o repite uno ya usado (poca probabilidad)
    if randint(0, 100) > 80:
      # repite
      auxPos = randint(0, auxPos)    
    # controla que no se pase (sino se resetea)
    if auxPos > len(fuenteRef):
      auxPos = 0
    # actualiza el diccionario y devuelve el valor que corresponde
    if diccReferenciasSecuenciales[auxIdRef] < auxPos:
      diccReferenciasSecuenciales[auxIdRef] = auxPos
    # devuelve la referencia que corresponda
    return fuenteRef[ auxPos ]

def agregar_referencia_documento(img_simbolo, nombreSimbolo):   
  global diccReferenciasSecuenciales
  global aux_anterior_referencia_documento
  
  # si se deben generar simbolos sin referencias
  if generar_simbolos_sin_referencias:
    tDoc = "   "
    numero_documento = -1
  else:
    if aux_anterior_referencia_documento!=None and len(aux_anterior_referencia_documento)==2:
      tDoc = aux_anterior_referencia_documento[0]
      numero_documento = aux_anterior_referencia_documento[1]
    else:
      # lista auxiliar de tipos de documentos ya creados
      auxRefDoc = []
      for k in diccReferenciasSecuenciales.keys():
        if "REF_DOC_" in k:
          p = k.find(".png_")
          if p < 0:
            p = 8
          else:
            p = p + 5
          auxRefDoc.append( k[p:] )

      # determina el tipo de documento al azar 
      # si reutiliza un documento ya usado o uno nuevo
      if len(auxRefDoc)>0 and randint(0, 100) > 80:  
        tDoc = choice( auxRefDoc )
      else:
        tDoc = devolverReferenciaSimbolo(nombreSimbolo)

      if usar_referencias_generadas == "AZAR":
        # determina número de referencia al azar
        numero_documento = randint(-1, 9)
      else:
        # determina número de referencia secuencial
        auxIdRef = "REF_DOC_" + nombreSimbolo + "_" + tDoc
        if auxIdRef in diccReferenciasSecuenciales:
          # utiliza último generado
          numero_documento =  diccReferenciasSecuenciales[auxIdRef]
          if numero_documento >= 0:
            # determina al azar si usa el incremento o repite uno ya usado (poca probabilidad)
            if randint(0, 100) > 80:
              # repite
              numero_documento = randint(0, numero_documento)   
            else:
              # incrementa el ultimo utilizado
              numero_documento = numero_documento + 1
              if numero_documento < 9:
                diccReferenciasSecuenciales[auxIdRef] = numero_documento
        else:      
          # determina al azar si usa nro de documento o no ("-1")
          if randint(0, 100) <= 90:  
            numero_documento = 0
          else:
            numero_documento = -1      
          diccReferenciasSecuenciales[auxIdRef] = numero_documento
    
      aux_anterior_referencia_documento = [tDoc, numero_documento]

  # agrega tipo documento en la imagen
  agregar_letras_documento(img_simbolo, tDoc)

  # agrega número documento en la imagen
  agregar_numero_documento(img_simbolo, numero_documento)


def agregar_letras_documento(img_simbolo, text):
  displacement = 0 if len(text) == 1 else 14
  blank_size = (40, 40) if len(text) == 1 else (47, 40)
  blank_position = (24 - displacement, 30)
  tamLetra = 25 if len(text) >= 3 else 28
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=tamLetra)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_documento(img_simbolo, numero):
  blank_size = (17, 23)
  blank_position = (45, 70)
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=20)
  if numero >= 0:
    text =  str(numero)
  else:
    text = "  " 
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)
  
def agregar_letras_conector_comienzo(img_simbolo):
  blank_size = (40, 45)
  blank_position = (37, 22)
  text = devolverReferenciaSimbolo("conector_comienzoCorte.png")
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=28)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_letras_conector_reanudacion(img_simbolo):
  blank_size = (40, 45)
  blank_position = (37, 40)
  text = devolverReferenciaSimbolo("conector_reanudacionCorte.png")  
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=28)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_proceso(img_simbolo):
  blank_size = (40, 40)
  blank_position = (40, 5)
  text = devolverReferenciaSimbolo("proceso.png")  
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=28)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_operacion_control(img_simbolo, esControl):
  blank_size = (40, 40)
  blank_position = (40, 30)
  if esControl:
    text = devolverReferenciaSimbolo("control.png")  
  else:
    text = devolverReferenciaSimbolo("operacion.png")  
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=28)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_decision(img_simbolo):
  blank_size = (50, 30)
  blank_position = (25, 40)
  number_size = (35, 37)
  number_position = (40, 30)
  text = devolverReferenciaSimbolo("decision.png")  
  blank = Image.new("RGB", blank_size, "white")
  number = Image.new("RGB", number_size, "white")
  draw = ImageDraw.Draw(number)
  font = ImageFont.truetype(font=path_fuente, size=28)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)
  img_simbolo.paste(number, number_position)

def agregar_numero_archivo_transitorio(img_simbolo):
  text = devolverReferenciaSimbolo("archivoTransitorio.png")  
  if len(text) == 1:
    blank_size = (25, 27)
    blank_position = (40, 8)
  elif len(text) == 2:
    blank_size = (28, 27)
    blank_position = (35, 8)
  elif len(text) >= 3:
    blank_size = (38, 27)
    blank_position = (31, 8)
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=25)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_archivo_definitivo(img_simbolo):
  text = devolverReferenciaSimbolo("archivoDefinitivo.png")  
  if len(text) == 1:
    blank_size = (25, 27)
    blank_position = (40, 30)
  elif len(text) == 2:
    blank_size = (28, 27)
    blank_position = (35, 30)
  elif len(text) >= 3:
    blank_size = (38, 27)
    blank_position = (31, 30)
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=25)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)


print("Funciones Especiales para Referencias de Símbolos  definidas.") 

In [ ]:
#@title Funciones de Generación de Símbolos

def agregar_simbolo(row, col, page, simbolo):    
  # obtiene símbolo a agregar
  img_simbolo = Image.open(join(path_simbolos, simbolo))
  # determina la posición a usar
  posicion = (int(col * box_size[0]) + int(0.5*(box_size[0] - img_simbolo.size[0])), int(row * box_size[1]) + int(0.5*(box_size[1] - img_simbolo.size[1])))
  # agrega la referencia en la imagen del símbolo
  if "documento" in simbolo:
    agregar_referencia_documento(img_simbolo, simbolo)
  elif "conector_comienzo" in simbolo:
    agregar_letras_conector_comienzo(img_simbolo)
  elif "conector_reanudacion" in simbolo:
    agregar_letras_conector_reanudacion(img_simbolo)
  elif "operacion" in simbolo or "control" in simbolo:
    agregar_numero_operacion_control(img_simbolo, ("control" in simbolo))
  elif "decision" in simbolo:
    agregar_numero_decision(img_simbolo)
    diccReferenciasSecuenciales["NO_REF_ULTIMA_POSICION_DECISION"] = posicion
  elif "archivoTransitorio" in simbolo:
    agregar_numero_archivo_transitorio(img_simbolo)  
  elif "archivoDefinitivo" in simbolo:
    agregar_numero_archivo_definitivo(img_simbolo)  
  elif "proceso" in simbolo and not 'procesoNoRelevado' in simbolo:
    agregar_numero_proceso(img_simbolo)
  # agrega el simbolo en la imagen
  page.paste(img_simbolo, posicion)

def seleccionarReglaSimbolo(reglasDisponibles, esSectorExterno=False):
  # si no hay nada para seleccionar, devuelve "None"
  if (reglasDisponibles is None) or (not reglasDisponibles) or (len(reglasDisponibles)==0):
    return None
  # desordena
  shuffle(reglasDisponibles)  
  if usar_config_probabilidad_simbolos:   
    # selecciona considerando las probabilidades de la configuración 
    inicios_weights = []
    if esSectorExterno:
      config_posProbab = config_posProbabUsarSectorExterno  
    else:
      config_posProbab = config_posProbabUsarSectorInterno  
    for r in reglasDisponibles:
      nombreSimbolo = r["destino"]
      if nombreSimbolo in config_simbolos.keys():
        inicios_weights.append( config_simbolos[nombreSimbolo][config_posProbab] )
      else:
        inicios_weights.append( config_Probab_Defecto )
    sel = choices(np.array(reglasDisponibles), weights=inicios_weights)[0]
  else:
    # selecciona al azar 
    sel = choice(np.array(reglasDisponibles))
  return sel

def agregar_simbolo_transicion(row, col, page, tipoTransicion, regla): 
  if "trasladoDeInformacion_horizontal" in tipoTransicion or "trasladoDeInformacion_vertical" in tipoTransicion:
      # determina posición origen y destino de la línea
      tamTransicion = dicc_Tamanio_Simbolos[ tipoTransicion ] 
      posicion_origen = (int(col * box_size[0]) + int(0.5*(box_size[0] - tamTransicion[0])), int(row * box_size[1]) + int(0.5*(box_size[1] - tamTransicion[1])))     
      # determina desplazamiento de la línea según simbolo de origen y destino
      desp_origen = agregar_simbolo_transicion_calcDesplSimbolo( regla["origen"] )
      desp_destino = agregar_simbolo_transicion_calcDesplSimbolo( regla["destino"] )
      # determina posiciones de acuerdo a tipo de transición
      if "vertical" in tipoTransicion:
          posicion_destino = (posicion_origen[0],  posicion_origen[1] + tamTransicion[1] + desp_destino[1] )
          posicion_origen = (posicion_origen[0], posicion_origen[1] - desp_origen[1])
      else:          
          if "decision" in regla["origen"] and diccReferenciasSecuenciales["NO_REF_ULTIMA_POSICION_DECISION"][0] > posicion_origen[0]:
            # se invierte origen y destino 
            # si es una transición a la izquierda de la Decisión
            despAuxi = desp_destino
            desp_destino = desp_origen
            desp_origen = despAuxi
          posicion_destino = (posicion_origen[0] + tamTransicion[0] + desp_destino[0],  posicion_origen[1] )      
          posicion_origen = (posicion_origen[0]- desp_origen[0], posicion_origen[1])
      # dibuja la línea
      draw = ImageDraw.Draw(page)
      draw.line([posicion_origen, posicion_destino], fill=(0, 0, 0), width=4)
      # devuelve posición para XML
      #print("###", tipoTransicion, posicion_origen, posicion_destino, tamTransicion, desp_origen, desp_destino, regla)
      return [posicion_origen, posicion_destino]
  else:
      agregar_simbolo(row, col, page, tipoTransicion)  
      return None

def agregar_simbolo_transicion_calcDesplSimbolo(simbolo):
  # calcula el desplazamiento que le tiene que hacer a la transición
  # en base al tamaño del simbolo
  if simbolo!="" and simbolo in dicc_Tamanio_Simbolos:
    desp = (int(0.5*(box_size[0] - dicc_Tamanio_Simbolos[ simbolo ][0])), int(0.5*(box_size[1] - dicc_Tamanio_Simbolos[ simbolo ][1])))
    return (0 if desp[0]<0 else desp[0], 0 if desp[1]<0 else desp[1])
  else:
    return (0, 0)

def elegir_inicio(reglas, externo):
  inicios = [regla for regla in reglas if regla["origen"] == ""]
  inicios = inicios if not externo else [inicio for inicio in inicios if not inicio["exclusivo_interno"]]
  inicios = inicios if posicion[1] == 0 else [inicio for inicio in inicios if not "conector" in inicio["destino"]]
  return seleccionarReglaSimbolo(inicios, externo)

def obtener_reglas_posibles(origen):
  global reglas, areas_generadas
  area_actual = areas_generadas[-1]
  reglas_posibles = [regla for regla in reglas if regla["origen"] == origen]
  reglas_posibles = reglas_posibles if not area_actual["externo"] else [regla for regla in reglas_posibles if not regla["exclusivo_interno"] ]
  reglas_posibles = reglas_posibles if generar_errores else [regla for regla in reglas_posibles if not regla["error"]]
  shuffle(reglas_posibles)
  return reglas_posibles

def elegir_accion(origen):
  global areas_generadas
  area_actual_externa = areas_generadas[-1]["externo"] 
  global posicion
  accion = None
  termino = False
  reglas_posibles = obtener_reglas_posibles(origen)
  
  while not termino:
    regla = seleccionarReglaSimbolo(reglas_posibles, area_actual_externa)
    ##regla = choice(np.array(reglas_posibles)) if reglas_posibles else None
    regla_aplicable, movimientos = puedo_aplicar_reglar(regla) if regla is not None else (False, [])

    if regla is not None and (not regla_aplicable or (posicion[1] == 0 and regla["destino"] == "")):
      reglas_posibles.remove(regla)
    else:
      accion = {
          "regla": regla,
          "movimientos": movimientos
      }
      termino = True
  
  return accion

def posiciones_usadas(posicion_transicion, posicion_simbolo):
  global areas_generadas

  area_actual = areas_generadas[-1]
  columna_otra_area = False
  posicion_usada = any(tuple(map(int, simbolo["posicion"])) == tuple(map(int, posicion_transicion)) or tuple(map(int, simbolo["posicion"])) == tuple(map(int, posicion_simbolo)) for simbolo in area_actual["simbolos"])

  if not posicion_usada:
    posicion_usada = any(tuple(map(round, simbolo["posicion"])) == tuple(map(round, posicion_transicion)) or tuple(map(round, simbolo["posicion"])) == tuple(map(round, posicion_simbolo)) for simbolo in area_actual["simbolos"])

  for area in areas_generadas:
    if area != area_actual:
      columna_otra_area = any(int(simbolo["posicion"][0]) == int(posicion_transicion[0]) or int(simbolo["posicion"][0]) == int(posicion_simbolo[0]) for simbolo in area["simbolos"])

      if not columna_otra_area:
        columna_otra_area = any(round(simbolo["posicion"][0]) == round(posicion_transicion[0]) or round(simbolo["posicion"][0]) == round(posicion_simbolo[0]) for simbolo in area["simbolos"])

      if columna_otra_area:
        puede_aplicar = False
        break

  return posicion_usada, columna_otra_area

def validar_posicion_usada(posicion_simbolo):
  global areas_generadas

  area_actual = areas_generadas[-1]
  columna_otra_area = False
  posicion_usada = any(tuple(map(int, simbolo["posicion"])) == tuple(map(int, posicion_simbolo)) for simbolo in area_actual["simbolos"])

  if not posicion_usada:
    posicion_usada = any(tuple(map(round, simbolo["posicion"])) == tuple(map(round, posicion_simbolo)) for simbolo in area_actual["simbolos"])

  for area in areas_generadas:
    if area != area_actual:
      columna_otra_area = any(int(simbolo["posicion"][0]) == int(posicion_simbolo[0]) for simbolo in area["simbolos"])

      if not columna_otra_area:
        columna_otra_area = any(round(simbolo["posicion"][0]) == round(posicion_simbolo[0]) for simbolo in area["simbolos"])

      if columna_otra_area:
        puede_aplicar = False
        break

  return posicion_usada, columna_otra_area

def puede_aplicar_decision(movimientos):
  global posicion, posicion_limite, areas_generadas

  caminos_decision = []
  posible_posicion = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
  posible_posicion = tuple(np.asarray(posible_posicion) + np.asarray(movimientos[1]))

  movimiento = (0, 1)
  posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
  posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
  posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

  puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[1] <= posicion_limite[1]
  if not puede_usar:
    movimiento = (choice([-1, 1]), 0)
    posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

    puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
    if puede_usar:
      caminos_decision.append(
          {
              "transicion": True,
              "simbolo": "transicion_dummy",
              "posicion": posicion_transicion
          }
      )
      caminos_decision.append(
          {
              "transicion": False,
              "simbolo": "simbolo_dummy",
              "posicion": posicion_simbolo
          }
      )

      movimiento = (-1, 0) if movimiento == (1, 0) else (1, 0)
      posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
      posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
      posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
      puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0

      if puede_usar:
        caminos_decision.append(
          {
              "transicion": True,
              "simbolo": "transicion_dummy",
              "posicion": posicion_transicion
          }
        )
        caminos_decision.append(
            {
                "transicion": False,
                "simbolo": "simbolo_dummy",
                "posicion": posicion_simbolo
            }
        )
  else:
    caminos_decision.append(
        {
            "transicion": True,
            "simbolo": "transicion_dummy",
            "posicion": posicion_transicion
        }
    )
    caminos_decision.append(
        {
            "transicion": False,
            "simbolo": "simbolo_dummy",
            "posicion": posicion_simbolo
        }
    )

    movimiento = (choice([-1, 1]), 0)
    posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

    puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
    if not puede_usar:
      movimiento = (-1, 0) if movimiento == (1, 0) else (1, 0)
      posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
      posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
      posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
      puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0

      if puede_usar:
        caminos_decision.append(
            {
                "transicion": True,
                "simbolo": "transicion_dummy",
                "posicion": posicion_transicion
            }
        )
        caminos_decision.append(
            {
                "transicion": False,
                "simbolo": "simbolo_dummy",
                "posicion": posicion_simbolo
            }
        )

    else:
      caminos_decision.append(
          {
              "transicion": True,
              "simbolo": "transicion_dummy",
              "posicion": posicion_transicion
          }
      )
      caminos_decision.append(
          {
              "transicion": False,
              "simbolo": "simbolo_dummy",
              "posicion": posicion_simbolo
          }
      )

  if puede_usar:
    areas_generadas[-1]["simbolos"].extend(caminos_decision)

  return puede_usar

def generar_copias_documento_operacion(regla, movimientos):
  global posicion, posicion_limite, areas_generadas
  #simbolo = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["posicion"] == posicion][0]
  #original = simbolo["copia"] == 0
  original = regla["copia"] == 0

  copias = randint(1, 3) if original else 1
  posicion_origen = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
  posicion_origen = tuple(np.asarray(posicion_origen) + np.asarray(movimientos[1]))
  copias_generadas = []

  movimiento_anterior = (0, 0)
  copia = 1
  desplazamiento = 1.5

  for i in range(1, copias):
    if movimiento_anterior != (0, 0):
      movimiento = (desplazamiento, 0) if movimiento_anterior == (-desplazamiento, 0) else (-desplazamiento, 0)
    else:
      movimiento = (choice([-desplazamiento, desplazamiento]), 0)

    posicion_simbolo = tuple(np.asarray(posicion_origen) + np.asarray(movimiento))

    posicion_usada, columna_otra_area = validar_posicion_usada(posicion_simbolo)
    puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
    
    if puede_usar:
      copias_generadas.append(
              {
                  "transicion": False,
                  "simbolo": regla["destino"] + "_copia",
                  "copia": copia,
                  "posicion": posicion_simbolo,
                  "movimiento": movimiento
              }
          )
      copia += 1
    
    movimiento_anterior = movimiento
    
  areas_generadas[-1]["simbolos"].extend(copias_generadas)

def puedo_aplicar_reglar(regla):
  global posicion, posicion_limite, areas_generadas

  simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["posicion"] == posicion]

  if len(simbolos) > 0 and "copia" in simbolos[0]:
    if regla["cambio_area"]:
      return False, []

    regla["copia"] = simbolos[0]["copia"]

    if "movimiento" in simbolos[0]:
      regla["movimiento"] = simbolos[0]["movimiento"]
  else:
    regla["copia"] = 0

  movimientos = []
  if "actualizacion" in regla["transicion"] or "consulta" in regla["transicion"]:
    movimiento = (1, 0)
    movimientos.append(movimiento)
    movimiento = (1, 0) if "actualizacion" in regla["transicion"] else (0, -1)
    movimientos.append(movimiento)

    posicion_transicion = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[1]))

    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
    puede_aplicar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0

    return puede_aplicar, movimientos

  if "decision" in regla["origen"]:
    area_actual = areas_generadas[-1]
    simbolos_decision = [simbolo for simbolo in area_actual["simbolos"] if "dummy" in simbolo["simbolo"]]
    movimiento = tuple(np.asarray(simbolos_decision[1]["posicion"]) - np.asarray(simbolos_decision[0]["posicion"]))
    if ("vertical" in regla["transicion"] and movimiento[1] == 1) or ("horizontal" in regla["transicion"] and abs(movimiento[0]) == 1):
        movimientos.append(movimiento)
        movimientos.append(movimiento)

        return True, movimientos

    return False, movimientos

  if "vertical" in regla["transicion"]:
    movimiento = (0, 1)
    movimientos.append(movimiento)
    movimientos.append(movimiento)

    posicion_transicion = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[1]))
    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
    puede_aplicar = (not posicion_usada) and posicion_simbolo[1] <= posicion_limite[1]

    if puede_aplicar and not generar_errores and "proceso" in regla["origen"]:
      area_actual = areas_generadas[-1]
      simbolo_anterior = area_actual["simbolos"][-3]["simbolo"] if len(area_actual["simbolos"]) >= 3 else ""
      puede_aplicar = False if "documento" in simbolo_anterior and "documento" in regla["destino"] else True

    if puede_aplicar and ("archivo" in regla["origen"] or "proceso" in regla["origen"]):
      area_actual = areas_generadas[-1]
      puede_aplicar = False if len(area_actual["simbolos"]) >= 3 and (not "temporalidad" in area_actual["simbolos"][-2]["simbolo"]) else True

    if puede_aplicar and "decision" in regla["destino"]:
      puede_aplicar = puede_aplicar_decision(movimientos)

    if puede_aplicar and "operacion" in regla["origen"]:
      generar_copias_documento_operacion(regla, movimientos)

    return puede_aplicar, movimientos

  if "horizontal" in regla["transicion"]:
    columna = choice([-1, 1])
    movimiento = (columna, 0)

    columnas_cruzar = 1
    if regla["cambio_area"]:
      area_actual = areas_generadas[-1]
      columnas_usadas = [simbolo["posicion"][0] for simbolo in area_actual["simbolos"] if not simbolo["transicion"]]
      columnas_cruzar = int(math.ceil(posicion[0] - min(columnas_usadas) if columna == -1 else max(columnas_usadas) - posicion[0])) + 1
      simbolos_usados = len([simbolo for simbolo in area_actual["simbolos"] if not simbolo["transicion"]])

      #es para evitar la transicion de area cuando hay un solo simbolo
      if simbolos_usados <= 1:
        return False, movimientos

    movimientos.append(movimiento)

    puede_aplicar = False
    posicion_prueba = posicion
    for i in range(columnas_cruzar):
      movimientos.append(movimiento)

      posicion_transicion = tuple(np.asarray(posicion_prueba) + np.asarray(movimientos[-2]))
      posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[-1]))
      posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

      puede_aplicar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
      posicion_prueba = posicion_simbolo

      if not puede_aplicar:
        break

    if not puede_aplicar:
      movimientos = []
      columna = 1 if columna == -1 else -1
      movimiento = (columna, 0)

      columnas_cruzar = 1
      if regla["cambio_area"]:
        area_actual = areas_generadas[-1]
        columnas_usadas = [simbolo["posicion"][0] for simbolo in area_actual["simbolos"] if not simbolo["transicion"]]
        columnas_cruzar = int(math.ceil(posicion[0] - min(columnas_usadas) if columna == -1 else max(columnas_usadas) - posicion[0])) + 1   

      movimientos.append(movimiento)

      posicion_prueba = posicion
      for i in range(columnas_cruzar):
        movimientos.append(movimiento)

        posicion_transicion = tuple(np.asarray(posicion_prueba) + np.asarray(movimientos[-2]))
        posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[-1]))
        posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

        puede_aplicar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
        posicion_prueba = posicion_simbolo

        if not puede_aplicar:
          break

    if puede_aplicar and "decision" in regla["destino"]:
      puede_aplicar = puede_aplicar_decision(movimientos)

    return puede_aplicar, movimientos

  return False, []

def agregar_transicion_copia(row, col, regla):
  global page, xml_annotation, uso_copia

  if regla["copia"] > 0 and "movimiento" in regla:

    movimiento = regla["movimiento"]
    draw = ImageDraw.Draw(page)
    posicion = (int(col * box_size[0]), int(row * box_size[1]))
    posicion_origen = (posicion[0]+50, posicion[1]-50)
    posicion_destino = (posicion[0]+50, posicion[1])
    draw.line([posicion_origen, posicion_destino], fill=(0, 0, 0), width=4)

    punto_origen = posicion_destino
    punto_destino = 0
    if movimiento[0] > 0:
      posicion_origen = (posicion[0]-100, posicion[1]-50)
      posicion_destino = (posicion[0]+50, posicion[1]-50)
      punto_destino = posicion_origen
    else:
      posicion_origen = (posicion[0]+50, posicion[1]-50)
      posicion_destino = (posicion[0]+200, posicion[1]-50)
      punto_destino = posicion_destino

    draw.line([posicion_origen, posicion_destino], fill=(0, 0, 0), width=4)
    del regla["movimiento"]
    agregar_anotacion_transicion_copia(punto_origen, punto_destino, xml_annotation)

    agregar_simbolo(row, col, page, regla["origen"])
    agregar_anotacion(row, col, xml_annotation, regla["origen"])
    uso_copia = True

def agregar_inicio(inicio):
  global posicion, page, xml_annotation
  agregar_simbolo(posicion[1], posicion[0], page, inicio["destino"])
  agregar_anotacion(posicion[1], posicion[0], xml_annotation, inicio["destino"])


def agregar_accion(accion):
  global posicion, page, xml_annotation, areas_generadas, posicion_limite
  regla = accion["regla"]
  movimientos = accion["movimientos"]

  agregar_transicion_copia(posicion[1], posicion[0], regla)
  posicion_original = posicion
  for movimiento in movimientos[:-1]:
    posicion = tuple(np.asarray(posicion) + np.asarray(movimiento))
    #agregar_simbolo(posicion[1], posicion[0], page, regla["transicion"])
    posTransicion = agregar_simbolo_transicion(posicion[1], posicion[0], page, regla["transicion"], regla)
    if posTransicion == None:
      agregar_anotacion(posicion[1], posicion[0], xml_annotation, regla["transicion"])
    else:
      agregar_anotacion_transicion_copia(posTransicion[0], posTransicion[1], xml_annotation, regla["transicion"])

    if "decision" in regla["origen"]:
      simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["simbolo"] == "transicion_dummy" and simbolo["posicion"] == posicion]
      simbolo = simbolos[0]
      simbolo["simbolo"] = regla["transicion"]
    else:
      areas_generadas[-1]["simbolos"].append(
          {
              "transicion": True,
              "simbolo": regla["transicion"],
              "posicion": posicion
          }
      )

  desplazamiento = 0
  if "consulta" in regla["transicion"]:
    desplazamiento = 0.5

  posicion = tuple(np.asarray(posicion) + np.asarray(movimientos[-1]))

  destino = regla["destino"]
  if posicion[1] >= posicion_limite[1] and not "archivo" in regla["destino"] and not "proceso" in regla["destino"] and not "destruccion" in regla["destino"] and not "decision" in regla["destino"]:
    destino = [item for item in reglas if item["destino"] == "" and "conector" in item["origen"]][0]["origen"]

  agregar_simbolo(posicion[1], posicion[0] + desplazamiento, page, destino)
  agregar_anotacion(posicion[1], posicion[0] + desplazamiento, xml_annotation, destino)

  if regla["cambio_area"]:
    areas_generadas.append(
        {
            "externo": not regla["exclusivo_interno"] and posicion[0] == 0,
            "simbolos": []
        }
    )

  if "decision" in regla["origen"]:
    simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["simbolo"] == "simbolo_dummy" and simbolo["posicion"] == posicion]
    simbolo = simbolos[0]
    simbolo["simbolo"] = regla["transicion"]
  else:
    areas_generadas[-1]["simbolos"].append(
        {
            "transicion": False,
            "simbolo": destino,
            "posicion": posicion
        }
    )

  if "actualizacion" in regla["transicion"] or "consulta" in regla["transicion"]:
    posicion = posicion_original

def dibujar_lineas_areas(areas):
  global page, xml_annotation
  draw = ImageDraw.Draw(page)

  posiciones_usadas = []
  for area in areas:
    area_interna = not area["externo"]
    primer_columna = min([simbolo["posicion"][0] for simbolo in area["simbolos"] if not simbolo["transicion"]])
    ultima_columna = max([simbolo["posicion"][0] for simbolo in area["simbolos"] if not simbolo["transicion"]])+1

    posicion_linea = int((0.5 + ultima_columna) * box_size[0])
    draw.line([(posicion_linea, 0), (posicion_linea, page.size[1])], fill=(0, 0, 0), width=2)

    if posicion_linea not in posiciones_usadas:
      posiciones_usadas.append(posicion_linea)
      agregar_anotacion_area(posicion_linea, xml_annotation, area_interna)

    posicion_linea = int((primer_columna - 0.5) * box_size[0])
    draw.line([(posicion_linea, 0), (posicion_linea, page.size[1])], fill=(0, 0, 0), width=2)

    if posicion_linea not in posiciones_usadas:
      posiciones_usadas.append(posicion_linea)
      agregar_anotacion_area(posicion_linea, xml_annotation, area_interna)

  for area in [area for area in areas if area["externo"]]:
    ultima_columna = max([simbolo["posicion"][0] for simbolo in area["simbolos"] if not simbolo["transicion"]])+1
    if area["externo"]:
      max_white_lines = int(page.size[1]/white_line_size)
      posicion_linea = int((0.5 + ultima_columna) * box_size[0])

      for i in range(max_white_lines):
        if i % 2 != 0:
          draw.line([(posicion_linea, i*white_line_size), (posicion_linea, i*white_line_size + white_line_size)], fill=(255, 255, 255), width=2)
  
  del draw

def agregar_temporalidad(posicion_maxima):
  global areas_generadas, page, posicion, reglas, xml_annotation

  posicion = tuple(np.asarray(posicion_maxima) + np.asarray((0, 1)))
  posicion_y_linea = int((0.5 + posicion[1]) * box_size[1])
  draw = ImageDraw.Draw(page)
  draw.line([(0, posicion_y_linea), (page.size[0], posicion_y_linea)], fill=(0, 0, 0), width=2)

  max_white_lines = int(page.size[0]/white_line_size)
  for i in range(max_white_lines):
    if i % 2 != 0:
      draw.line([(i*white_line_size, posicion_y_linea), (i*white_line_size + white_line_size, posicion_y_linea)], fill=(255, 255, 255), width=2)

  usar_label = True if not generar_errores else choice([True, False])
  if usar_label:
    text = devolverReferenciaSimbolo("temporalidad.png")
    font = ImageFont.truetype(font=path_fuente, size=20)
    draw.text((5, posicion_y_linea-30), text,(0,0,0), font=font) # siempre va a la izquierda
    #draw.text((page.size[0]/2, posicion_y_linea-30), text,(0,0,0), font=font)
  else:
    marcar_documento_error()

  areas_generadas[-1]["simbolos"].append(
      {
        "transicion": False,
        "simbolo": "temporalidad",
        "posicion": (0, int((0.5 + posicion[1])))
      }
  )

  agregar_anotacion_temporalidad(posicion_y_linea-20, xml_annotation)

  posicion = tuple(np.asarray(posicion) + np.asarray((0, 1)))
  inicio = elegir_inicio(reglas, areas_generadas[-1]["externo"])
  agregar_inicio(inicio)
  origen = inicio["destino"]
  areas_generadas[-1]["simbolos"].append(
    {
        "transicion": False,
        "simbolo": origen,
        "posicion": posicion
    }
  )

  return origen

print("Funciones de Generación de Símbolos definidas.")

5) Generar los Casos de Cursograma con reglas:

In [ ]:
#@title Llevar a cabo la Generación de los Casos

# inicializar carpeta destino
inicializar_carpeta_destino(path_destino, (forzar_generar_combinaciones_parametros or forzar_drive_borrado))

# inicializar variables globales
reglas = cargar_reglas()
posicion_limite = (int(page_size[0]/box_size[0])-1, int(page_size[1]/box_size[1])-1)
dicc_Tamanio_Simbolos = obtener_tamanio_simbolos()

print("** Comienza Generación **")

if forzar_generar_combinaciones_parametros:
  # indica la lista de casos a generar
  # (las letras indica el tipo de configuración - ver abajo)
  lista_config_tipos_casos = [ 'SARV', 'SARA', 'SARC', 'SARS', 'SPRV', 'SPRA', 'SPRC', 'SPRS', 'SFRV', 'SFRS', 'SFRA' ]
  # determina cantida de casos
  cantidad_casos = cantidad_casos_generar_total // len(lista_config_tipos_casos) 
  if cantidad_casos > 999:
    cantidad_casos = 999
else:
  lista_config_tipos_casos = [ '-' ]
  cantidad_casos = cantidad_casos_generar_total

# comienza a ciclar
for tipo_caso in lista_config_tipos_casos:
  # si determina los parámetros para cada tipo de combinación
  if tipo_caso == '-':
    prefijoNombreCaso = prefijo_inicial_caso_generado
  else:
    prefijoNombreCaso = tipo_caso
    # limpia los parámetros
    generar_errores = False        
    # muestra tipo de combinación
    tipo_caso_genSimbolos = tipo_caso[1:2]   
    tipo_caso_genReferencias = tipo_caso[3:4]
    print("> Combinación tipo ", tipo_caso, " (símbolos ", tipo_caso_genSimbolos, ", & referencias ", tipo_caso_genReferencias, "):")
    # determina parámetros de generación de símbolos 
    if tipo_caso_genSimbolos == 'F':
      # símbolos forzados (al azar)
      usar_config_probabilidad_simbolos = False
      forzar_decision = True
      forzar_copia = True
      forzar_area_externa = True
      forzar_actualizacion = True  
    elif tipo_caso_genSimbolos == 'P':
      # símbolos considerando probabilidades de configuración    
      usar_config_probabilidad_simbolos = True
      forzar_decision = False 
      forzar_copia = False 
      forzar_area_externa = False 
      forzar_actualizacion = False       
    else: #elif tipo_caso_genSimbolos == 'A':
      # símbolos al azar
      usar_config_probabilidad_simbolos = False
      forzar_decision = False 
      forzar_copia = False 
      forzar_area_externa = False 
      forzar_actualizacion = False       
    # determina parámetros de generación de referencias de símbolos    
    if tipo_caso_genReferencias == 'V':
      # sin referencias (vacías)
      generar_simbolos_sin_referencias = True 
      usar_config_valores_tipo_referencias = False
      usar_referencias_generadas = "AZAR"       
    elif tipo_caso_genReferencias == 'S':
      # referencias por configuración secuenciales
      generar_simbolos_sin_referencias = False
      usar_config_valores_tipo_referencias = True
      usar_referencias_generadas = "SECUENCIAL_TIPO_REFERENCIA"       
    elif tipo_caso_genReferencias == 'C':
      # referencias por configuración al azar
      generar_simbolos_sin_referencias = False      
      usar_config_valores_tipo_referencias = True
      usar_referencias_generadas = "AZAR" 
    else: # elif tipo_caso_genReferencias == 'A':
      # con referencias totalmente al azar
      generar_simbolos_sin_referencias = False
      usar_config_valores_tipo_referencias = False
      usar_referencias_generadas = "AZAR" 

  # lleva a cabo la generación de casos 
  # considerando los parámetros
  i = 1
  while i <= cantidad_casos:

    # determina nombre del caso
    nombre = "curso_" + prefijoNombreCaso + "_" + ("000000" + str(i))[-6:]
    print(" - Generando #", i, " de ", cantidad_casos, " - ", nombre)
    
    # inicializa imagen y XML del caso
    page, xml_annotation = crear_cursograma(nombre)

    # inicializa diccionario para referencias
    diccReferenciasSecuenciales = {}

    # comienza la generación
    esInicioAreaExterna = False
    if forzar_area_externa:
      # si se fuerza area externa, ya la indica
      posicion = (0, 0)
      esInicioAreaExterna = True
    else:
      posicion = (randint(0, posicion_limite[0]), 0)
      if posicion[0] == 0:
        #determina al azar si va a ser area externa 
        if randint(0,100) > 50:
          esInicioAreaExterna = True

    inicio = elegir_inicio(reglas, esInicioAreaExterna)
    agregar_inicio(inicio)
    origen = inicio["destino"]
    areas_generadas = [
      {
          "externo":  esInicioAreaExterna,
          "simbolos": [
              {
                  "transicion": False,
                  "simbolo": origen,
                  "posicion": posicion
              }
          ]
      }
    ]
    # continua la generación
    decisiones = []
    termino = False
    uso_decision = False
    uso_copia = False
    uso_area_externa = False
    uso_actualizacion = False
    while not termino:
      
      accion = elegir_accion(origen)

      if accion["regla"] is not None and accion["regla"]["destino"] != "":
        if accion["regla"]["error"]:
          marcar_documento_error()
        
        if "flecha" in accion["regla"]["transicion"]:
          uso_actualizacion = True

        agregar_accion(accion)
        origen = accion["regla"]["origen"] if "actualizacion" in accion["regla"]["transicion"] or "consulta" in accion["regla"]["transicion"] else accion["regla"]["destino"]

        if "decision" in origen:
          uso_decision = True

        if posicion[1] >= posicion_limite[1] and not "archivo" in accion["regla"]["destino"] and not "proceso" in accion["regla"]["destino"] and not "destruccion" in accion["regla"]["destino"]:
          origen = [item for item in reglas if item["destino"] == "" and "conector" in item["origen"]][0]["origen"]
        
      else:
          areas_con_decisiones = [area for area in areas_generadas if any([simbolo for simbolo in area["simbolos"] if "dummy" in simbolo["simbolo"]])]
          areas_con_copias = [area for area in areas_generadas if any([simbolo for simbolo in area["simbolos"] if "_copia" in simbolo["simbolo"]])]

          if len(areas_con_decisiones) > 0:
            area_con_decision = areas_con_decisiones[0]
            simbolos_decision = [simbolo for simbolo in area_con_decision["simbolos"] if "dummy" in simbolo["simbolo"]]
            
            movimiento_realizado = np.asarray(simbolos_decision[1]["posicion"]) - np.asarray(simbolos_decision[0]["posicion"])
            posicion = tuple(np.asarray(simbolos_decision[0]["posicion"]) - movimiento_realizado)

            origen = "decision.png"
            areas_generadas.remove(area_con_decision)
            areas_generadas.append(area_con_decision)

          if len(areas_con_copias) > 0:
            area_con_copias = areas_con_copias[0]
            simbolo_copia = [simbolo for simbolo in area_con_copias["simbolos"] if "_copia" in simbolo["simbolo"]][0]
            posicion = simbolo_copia["posicion"]
            origen = simbolo_copia["simbolo"].split("_copia")[0]
            simbolo_copia["simbolo"] = origen
          
          if len(areas_con_decisiones) == 0 and len(areas_con_copias) == 0:
            y_maximo = -1
            area_con_maximo = None
            posicion_con_maximo = None
            ultimo_simbolo = None
            for area in areas_generadas:
              for simbolo in area["simbolos"]:
                posicion_area = simbolo["posicion"]

                if posicion_area[1] > y_maximo:
                  area_con_maximo = area
                  y_maximo = posicion_area[1]
                  posicion_con_maximo = posicion_area
                  ultimo_simbolo = simbolo["simbolo"]

            if y_maximo < posicion_limite[1]-2 and not "conector" in ultimo_simbolo:
              areas_generadas.remove(area_con_maximo)
              areas_generadas.append(area_con_maximo)
              origen = agregar_temporalidad(posicion_area)
            else:
              termino = True

    uso_area_externa = any([area for area in areas_generadas if area["externo"]])

    if (not forzar_decision and not forzar_copia and not forzar_area_externa and not forzar_actualizacion) or (forzar_decision and uso_decision) or (forzar_copia and uso_copia) or (forzar_area_externa and uso_area_externa) or (forzar_actualizacion and uso_actualizacion):
      # cierra el caso y lo graba 
      dibujar_lineas_areas(areas_generadas)
      guardar_cursograma(nombre, page, xml_annotation)
      i += 1
    else:
      # indica por qué se descarta el caso
      if (forzar_decision and not uso_decision):
        print("\t descartado porque no tiene Decisión!")
      if (forzar_copia and not uso_copia):
        print("\t descartado porque no usa Copias de Documentos!")
      if (forzar_area_externa and not uso_area_externa):
        print("\t descartado porque no tiene Área Externa!")
      if (forzar_actualizacion and not uso_actualizacion):
        print("\t descartado porque no tiene Actualización!")

print("** Generación Terminada **")

if forzar_drive_actualizar:
  print("\n** Forzando la actualización del drive **")
  # Fuerza la actualizacion del drive
  drive.flush_and_unmount()
  # vuelve a montar
  drive.mount('/content/gdrive', force_remount=True)
  print("**Actualización del drive terminada **")

contar_cantidad_casos(path_destino)
